<a href="https://colab.research.google.com/github/LVaditya/BusinessCard-Reader/blob/main/BusinessCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr 

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [4]:
from pyngrok import ngrok

In [5]:
ngrok.set_auth_token('2L6Kz7Hvawi3QB7akHGUu8CRsfi_5JPuFqERA67quEKGFuvaU')

In [45]:
%%writefile app.py

# import all the packages needed

import streamlit as st 
from PIL import Image
import easyocr
import pandas as pd
import numpy as np
from numpy import asarray
import re
import os
import sqlite3


conn = sqlite3.connect('Businesscard.db')
c = conn.cursor()


#-------Page Configuration--------

st.set_page_config(page_title="Business Card Reader",
                    page_icon=":black_small_square:",
                    layout="wide"
                    )

st.title("Business Card Reader")


#loading the image


def main():
 
  #file to be uploaded is taken as image_file

  image_file= st.file_uploader("upload the image of business card you want to extract data", type=["jpg","png","jpeg"])


#-------image file is read using OCR------
  @st.cache_data
  def load_model():
    reader=easyocr.Reader(['en'])
    return reader
  
  reader=load_model()

  #-----------------if the image is uploaded------

  if image_file is not None:

    img = Image.open(image_file)  #read image
    st.image(img, width=400)                 #display image

    #saving file into tempDir

    with open('tempDir.png','wb') as f:     
      f.write(image_file.getbuffer())

    #-------converting file to binary array---------

   
    with st.spinner("Please wait.... work in progress"):

      data = reader.readtext(np.array(img))       #load the data from image as data
      text_result=[]

      text_dict={'Name':[], 'Designation':[], 'Phone':[], 'Email':[], 'Website':[], 'City':[], 'State':[], 'Pincode':[]}
      for text in data:
        text_result.append(text[1])

      text_dict['Name'] = text_result[0]
      text_dict['Designation']= text_result[1]


      for text in text_result:
        if 'WWW'in text or 'www' in text or 'wwW' in text:
          text_dict['Website'] = text

        elif '@' in text: 
          text_dict['Email']= text

        elif '-' in text:
          text_dict['Phone'] = text
        
        else:
          l=list(map(str,text.split(" ")))
          for i in l:
            i=i.replace(",","")
            i=i.replace(";","")
            if i.isdigit():
              text_dict['Pincode'] = i
            elif i.lower() in ['tamilnadu', 'kerala','karnataka', 
            'maharashtra', 'delhi', 'andhra pradesh', 'telangana' ]:
              text_dict['State'] = i
            elif i.lower() in ['chennai', 'hydrabad', 'salem', 'erode', 'tirupur', 'coimbatore']:
              text_dict['City'] = i

      text_dict_1=st.experimental_data_editor(data=text_dict)            #All collected data is displayed in table format

      
      with open("tempDir.png",'rb') as m:   # reading image as binary data and storing  
        img_data = m.read()                 # in img_data


      text_dict_1['image'] = img_data

      df=pd.DataFrame(text_dict_1, index=[0])    #converting into dataframe

#-----------to upload the data into sqlite------

    upload = st.button("upload data")

    if upload:

      try:
        df.to_sql('Card_values',conn,if_exists='append',index=False)    # data appended into a table in database
        


      except:
        print("connection error")

    conn.commit()
    c.close()
    conn.close()

#----------if image not uploaded-----
    
  else:
    st.write("upload an image of business card")



  
if __name__=='__main__':
  main()

Overwriting app.py


In [7]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://933e-34-125-24-137.ngrok.io" -> "http://localhost:80"


In [13]:
import sqlite3
import pandas as pd

In [25]:
conn=sqlite3.connect('Businesscard.db')
c=conn.cursor()


In [57]:
query='''select * from Card_values'''

In [58]:
df_new = pd.read_sql(query,conn)

In [59]:
df_new

,Name,Designation,Phone,Email,Website,City,State,Pincode,image
0,KARTHICK,General Manager,+123-456-7890,hello@Borcelle.com,wwW.Borcelle.com,Salem,TamilNadu,6004513,None
1,KARTHICK,General Manager,+123-456-7890,hello@Borcelle.com,wwW.Borcelle.com,Salem,TamilNadu,6004513,None
2,SANTHOSH,Technical Manager,+123-456-1234,hello@Sun.com,www.Suncom,Tirupur,TamilNadu,641603,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
3,KARTHICK,General Manager,+123-456-7890,hello@Borcelle.com,www.Borcelle.com,Salem,TamilNadu,6004513,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [60]:
sql1 = '''select * from Card_values where Website = 'www.Borcelle.com' '''

In [61]:
m= c.execute(sql1)

In [62]:
for x in m:
  rec_data=(x[8])

In [ ]:
rec_data

In [63]:
with open('card_image.png','wb') as f:
  f.write(rec_data)